In [ ]:

# === IMPORTS E SETUP ===
import os
import sys
import json
import torch
import numpy as np
import pandas as pd

# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Adicionar path para imports
sys.path.append('/content/drive/MyDrive/enem_tcc_resultados')

# Imports das funções utilitárias
from utils import load_enem_dataset, calcular_resultados, apply_protocol, PROTOCOL_LABELS, PROTOCOL_ORDER, generate_latex_table
from model_utils import (
    run_grid_search_all_competencies,
    train_final_models_all_competencies,
    evaluate_final_models,
    get_device
)
from config import setup_colab_paths, MODEL_TEMPLATES, TOKENIZER_NAMES, COMPETENCIES, TEST_YEARS, TRAIN_YEARS

# Setup paths
DRIVE_BASE_PATH = setup_colab_paths()
print(f"✓ Google Drive montado. Resultados serão salvos em: {DRIVE_BASE_PATH}")

# Configurações do modelo
MODEL_NAME_TEMPLATE = MODEL_TEMPLATES["jbsc_finetuned_by_comp"]["bertimbau"]
TOKENIZER_NAME = TOKENIZER_NAMES.get("bertimbau", None)
MAX_LEN = 512
device = get_device()
print(f"Using device: {device}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ Google Drive montado. Resultados serão salvos em: /content/drive/MyDrive/enem_tcc_resultados


In [ ]:
# === CARREGAR DATASET ===
df_train, df_test = load_enem_dataset(
    dataset_name="laisnuto/self-collected-ENEM-dataset",
    split="train",
    anos_teste=TEST_YEARS
)


In [ ]:
# === HYPERPARAMETER SEARCH ===
hyperparameter_space = {
    'learning_rate': [1e-5],
    'batch_size': [16, 32],
    'epochs': [8, 12, 16]
}

print("Starting hyperparameter search...")
print(f"Search space: {hyperparameter_space}")

# Create directory for saving results
SAVE_DIR = os.path.join(DRIVE_BASE_PATH, "fine_tuning_modelos_jbsc", "bertimbau_finetuned_by_comp")
os.makedirs(SAVE_DIR, exist_ok=True)
print(f"✓ Diretório de salvamento: {SAVE_DIR}")

checkpoint_file = os.path.join(SAVE_DIR, "hyperparameter_search_checkpoint.json")
final_results_file = os.path.join(SAVE_DIR, "hyperparameter_search_results_bertimbau.json")

# Run grid search for all competencies
best_hyperparams, best_qwk_scores = run_grid_search_all_competencies(
    df_train=df_train,
    hyperparameter_space=hyperparameter_space,
    model_name_template=MODEL_NAME_TEMPLATE,
    tokenizer_name=TOKENIZER_NAME,
    competencies=COMPETENCIES,
    max_len=MAX_LEN,
    device=device,
    checkpoint_file=checkpoint_file
)

# Save final results
results = {
    'best_hyperparams': best_hyperparams,
    'best_qwk_scores': best_qwk_scores
}
with open(final_results_file, 'w') as f:
    json.dump(results, f, indent=2)

print(f"\n✓ Final results saved to {final_results_file}")

# Print summary
print("\n=== HYPERPARAMETER SEARCH RESULTS ===")
for comp in COMPETENCIES:
    comp_key = f'C{comp}'
    if comp_key in best_hyperparams:
        print(f"{comp_key}: {best_hyperparams[comp_key]} -> QWK: {best_qwk_scores[comp_key]:.3f}")
    else:
        print(f"{comp_key}: NOT COMPLETED")



Carregando o dataset...
Tamanho do dataset: 157 linhas e 4 colunas
Distribuição por ano:
ano
2016     1
2018    15
2019    31
2020    29
2021    29
2022    16
2023    11
2024    25
Name: count, dtype: Int64
Total: 157
Anos no treino: [2019, 2020, 2021, 2024]
Anos no teste : [2016, 2018, 2022, 2023]
Tamanho treino/teste: 114 / 43
Test percentage: 27.4%
Train size: 114, Test size: 43
Test percentage: 27.4%


In [ ]:
# === TRAIN FINAL MODELS WITH BEST HYPERPARAMETERS ===
# Load best hyperparameters
final_results_file = os.path.join(SAVE_DIR, "hyperparameter_search_results_bertimbau.json")
try:
    with open(final_results_file, 'r') as f:
        search_results = json.load(f)
    best_hyperparams = search_results['best_hyperparams']
    print(f"✓ Loaded hyperparameter search results from {final_results_file}")
except FileNotFoundError:
    print(f"⚠️ No hyperparameter search results found. Using defaults.")
    best_hyperparams = {f'C{c}': {'learning_rate': 2e-5, 'batch_size': 16, 'epochs': 5} for c in COMPETENCIES}

# Train final models
tokenizers_final, models_final = train_final_models_all_competencies(
    df_train=df_train,
    best_hyperparams=best_hyperparams,
    model_name_template=MODEL_NAME_TEMPLATE,
    tokenizer_name=TOKENIZER_NAME,
    competencies=COMPETENCIES,
    save_dir=SAVE_DIR,
    model_name_prefix="bertimbau_jbsc",
    max_len=MAX_LEN,
    device=device
)

print("\n=== Final models training completed ===")
print("Models saved in:", SAVE_DIR)


Using device: cuda
Starting hyperparameter search...
Search space: {'learning_rate': [1e-05], 'batch_size': [16, 32], 'epochs': [8, 12, 16]}
✓ Diretório de salvamento: /content/drive/MyDrive/enem_tcc_resultados/fine_tuning_modelos_jbsc/bertimbau_finetuned_by_comp
Loaded checkpoint from /content/drive/MyDrive/enem_tcc_resultados/fine_tuning_modelos_jbsc/bertimbau_finetuned_by_comp/hyperparameter_search_checkpoint.json
Found saved results for: ['C1', 'C2', 'C3', 'C4', 'C5']


In [ ]:
# === FINAL TEST EVALUATION ===
out_final = evaluate_final_models(
    df_test=df_test,
    tokenizers_final=tokenizers_final,
    models_final=models_final,
    competencies=COMPETENCIES,
    max_len=MAX_LEN,
    device=device,
    save_csv_path=os.path.join(SAVE_DIR, "predicoes_bertimbau_final_hyperopt.csv")
)

# Evaluate on test set
print("\n=== FINAL TEST SET EVALUATION ===")
test_results = {}
for c in COMPETENCIES:
    comp_key = f"C{c}"
    pred_col = f"pred_{comp_key}"
    
    pares = out_final[[comp_key, pred_col]].dropna()
    if pares.empty:
        print(f"⚠️ No valid data for {comp_key}")
        continue
    
    y_real = pares[comp_key].astype(int).tolist()
    y_pred = pares[pred_col].astype(int).tolist()
    
    resultado = calcular_resultados(y_real, y_pred, is_final=True)
    test_results[comp_key] = resultado
    
    print(f"\n🔎 Test Results - {comp_key}")
    print(f"  Samples: {len(y_real)}")
    print(f"  QWK     : {resultado['QWK']:.3f}")
    print(f"  F1 Macro: {resultado['F1-Macro']:.3f}")
    print(f"  F1 Wghtd: {resultado['F1-Weighted']:.3f}")
    print(f"  ACC     : {resultado['ACC']:.3f}")
    print(f"  RMSE    : {resultado['RMSE']:.2f}")

# Summary
print("\n=== FINAL TEST SET SUMMARY ===")
qwk_scores = [test_results[f"C{c}"]["QWK"] for c in COMPETENCIES if f"C{c}" in test_results]
if qwk_scores:
    print(f"Average QWK across competencies: {np.mean(qwk_scores):.3f}")
    print(f"Best QWK: {max(qwk_scores):.3f}")
    print(f"Worst QWK: {min(qwk_scores):.3f}")


C1 already completed. Skipping.
  Best params: {'learning_rate': 1e-05, 'batch_size': 16, 'epochs': 12}
  Best QWK: 0.323


In [ ]:
# === EVALUATE ALL PROTOCOLS ===
print("=== AVALIANDO TODOS OS PROTOCOLOS COM MODELOS FINAIS ===")
avaliacoes_por_modelo = {"bertimbau-final": {}}

for protocol_key in PROTOCOL_ORDER:
    print(f"\n=== Esquema: {PROTOCOL_LABELS[protocol_key]} ===")
    avaliacoes = {}
    
    for c in COMPETENCIES:
        comp_key = f"C{c}"
        pred_col = f"pred_{comp_key}"
        
        pares = out_final[[comp_key, pred_col]].dropna()
        if pares.empty:
            continue
        
        y_real = pares[comp_key].astype(int).tolist()
        y_pred = pares[pred_col].astype(int).tolist()
        
        # Apply protocol
        y_r, y_p = apply_protocol(y_real, y_pred, protocol_key)
        
        if not y_r:
            continue
        
        resultado = calcular_resultados(y_r, y_p, is_final=False)
        avaliacoes[comp_key] = resultado
        
        print(f"\n🔎 Avaliação - {comp_key}")
        print(f"  QWK               : {resultado['QWK']:.3f}")
        print(f"  F1 Macro          : {resultado['F1-Macro']:.3f}")
        print(f"  F1 Weighted       : {resultado['F1-Weighted']:.3f}")
    
    avaliacoes_por_modelo["bertimbau-final"][protocol_key] = avaliacoes

# Generate LaTeX tables
print("\n=== TABELAS LaTeX ===")
metrics = [("QWK", "QWK"), ("F1-Macro", "F1 Macro"), ("F1-Weighted", "F1 Weighted")]

for met_key, met_title in metrics:
    tex = generate_latex_table(
        avaliacoes_por_modelo=avaliacoes_por_modelo,
        model_key="bertimbau-final",
        metric_key=met_key,
        metric_title=f"{met_title} por competência para o modelo BERTimbau com hyperparameter optimization",
        competencias=COMPETENCIES
    )
    print(f"\n=== Tabela LaTeX — {met_title} — bertimbau-final ===\n")
    print(tex)


C2 already completed. Skipping.
  Best params: {'learning_rate': 1e-05, 'batch_size': 16, 'epochs': 12}
  Best QWK: 0.504


C3 already completed. Skipping.
  Best params: {'learning_rate': 1e-05, 'batch_size': 32, 'epochs': 12}
  Best QWK: 0.380


C4 already completed. Skipping.
  Best params: {'learning_rate': 1e-05, 'batch_size': 32, 'epochs': 12}
  Best QWK: 0.490


C5 already completed. Skipping.
  Best params: {'learning_rate': 1e-05, 'batch_size': 16, 'epochs': 16}
  Best QWK: 0.509



=== HYPERPARAMETER SEARCH RESULTS ===
C1: {'learning_rate': 1e-05, 'batch_size': 16, 'epochs': 12} -> QWK: 0.323
C2: {'learning_rate': 1e-05, 'batch_size': 16, 'epochs': 12} -> QWK: 0.504
C3: {'learning_rate': 1e-05, 'batch_size': 32, 'epochs': 12} -> QWK: 0.380
C4: {'learning_rate': 1e-05, 'batch_size': 32, 'epochs': 12} -> QWK: 0.490
C5: {'learning_rate': 1e-05, 'batch_size': 16, 'epochs': 16} -> QWK: 0.509

✓ Final results saved to /content/drive/MyDrive/enem_tcc_resultados/fine_tuning_modelos_jbsc/bertimbau_finetuned_by_comp/hyperparameter_search_results_bertimbau.json
✓ Checkpoint saved to /content/drive/MyDrive/enem_tcc_resultados/fine_tuning_modelos_jbsc/bertimbau_finetuned_by_comp/hyperparameter_search_checkpoint.json


✓ Diretório de salvamento: /content/drive/MyDrive/enem_tcc_resultados/fine_tuning_modelos_jbsc/bertimbau_finetuned_by_comp
✓ Loaded hyperparameter search results from /content/drive/MyDrive/enem_tcc_resultados/fine_tuning_modelos_jbsc/bertimbau_finetuned_by_comp/hyperparameter_search_results_bertimbau.json
  Found hyperparameters for: ['C1', 'C2', 'C3', 'C4', 'C5']
Training final models with best hyperparameters...

=== Training Final Model — Competência C1 ===
[C1] Carregando kamel-usp/jbcs2025_bertimbau_base-C1
[C1] Tamanho treino: 114 | Batches: 8
[C1] Hyperparameters: {'learning_rate': 1e-05, 'batch_size': 16, 'epochs': 12}


/tmp/ipython-input-2920096088.py:66: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=(device.type == "cuda"))


[C1] Epoch 1/12 (final):   0%|          | 0/8 [00:00<?, ?it/s]

/tmp/ipython-input-2920096088.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(device.type == "cuda")):


[C1] epoch 1/12 - train loss: 1.1518 | tempo: 4.2s


...

[C5] epoch 14/16 - train loss: 0.6699 | tempo: 3.9s


[C5] Epoch 15/16 (final):   0%|          | 0/8 [00:00<?, ?it/s]

[C5] epoch 15/16 - train loss: 0.6168 | tempo: 3.9s


[C5] Epoch 16/16 (final):   0%|          | 0/8 [00:00<?, ?it/s]

[C5] epoch 16/16 - train loss: 0.7195 | tempo: 3.5s
[C5] ✓ Modelo final salvo em: /content/drive/MyDrive/enem_tcc_resultados/fine_tuning_modelos_jbsc/bertimbau_finetuned_by_comp/bertimbau_jbsc_C5_finetuned_com_redacoes_oficiais

=== Final models training completed ===
Models saved in: /content/drive/MyDrive/enem_tcc_resultados/fine_tuning_modelos_jbsc/bertimbau_finetuned_by_comp


=== FINAL TEST EVALUATION ===
Test set size: 43
Test set years: [np.int64(2016), np.int64(2018), np.int64(2022), np.int64(2023)]

Making predictions with final models...
Predicting C1...
Predicting C2...
Predicting C3...
Predicting C4...
Predicting C5...
✓ Final predictions saved to: /content/drive/MyDrive/enem_tcc_resultados/fine_tuning_modelos_jbsc/bertimbau_finetuned_by_comp/predicoes_bertimbau_final_hyperopt.csv

=== FINAL TEST SET EVALUATION ===
Evaluating on held-out test set (never used for training or hyperparameter tuning)

🔎 Test Results - C1
  Samples: 43
  QWK     : 0.393
  F1 Macro: 0.217
  F1 Wghtd: 0.359
  ACC     : 0.465
  RMSE    : 21.99

🔎 Test Results - C2
  Samples: 43
  QWK     : 0.355
  F1 Macro: 0.213
  F1 Wghtd: 0.591
  ACC     : 0.628
  RMSE    : 44.09

🔎 Test Results - C3
  Samples: 43
  QWK     : 0.453
  F1 Macro: 0.139
  F1 Wghtd: 0.183
  ACC     : 0.256
  RMSE    : 30.95

🔎 Test Results - C4
  Samples: 43
  QWK     : 0.658
  F1 Macro: 0.256
  F1 Wghtd: 0.39

=== AVALIANDO TODOS OS PROTOCOLOS COM MODELOS FINAIS ===

📊 Avaliando modelo: bertimbau-final (com hyperparameter optimization)

=== Esquema: Sem ajuste de escala ===

🔎 Avaliação - C1
  QWK               : 0.393
  F1 Macro          : 0.217
  F1 Weighted       : 0.359

🔎 Avaliação - C2
  QWK               : 0.355
  F1 Macro          : 0.213
  F1 Weighted       : 0.591

🔎 Avaliação - C3
  QWK               : 0.453
  F1 Macro          : 0.139
  F1 Weighted       : 0.183

🔎 Avaliação - C4
  QWK               : 0.658
  F1 Macro          : 0.256
  F1 Weighted       : 0.391

🔎 Avaliação - C5
  QWK               : 0.385
  F1 Macro          : 0.199
  F1 Weighted       : 0.368

=== Esquema: Correção dupla (baixo/cima) ===

🔎 Avaliação - C1
  QWK               : 0.331
  F1 Macro          : 0.309
  F1 Weighted       : 0.601

🔎 Avaliação - C2
  QWK               : 0.350
  F1 Macro          : 0.323
  F1 Weighted       : 0.630

🔎 Avaliação - C3
  QWK               : 0.411
  F1 Macro          : 0.278